In [12]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df = pd.read_excel('에타 1차 라벨링.xlsx'); df

,Unnamed: 0,date,likes,scrapes,comments,merged,label
0,0,2024-05-13 00:00:00,0,0,"[{'comment': '장게로 가세요', 'likes': 0}]",❤️컴사코 노해선 교수님 족보 정리본 + 과제 코딩 답안 팝니다 ☑️2023학년도 ...,0
1,1,2024-05-13 00:00:00,0,0,"[{'comment': '교바교', 'likes': 0}]",영발 지각 지각하면 점수 많이 깎이나요…?,0
2,2,2024-05-13 00:00:00,0,0,[{'comment': '얼마나 늦었는데? 교바교긴 한데 대체로 하루 이내면 신경 ...,창글 과제 제출? 간단하게 하는 거 늦었는데 A+가능..?ㅠㅠㅠ,1
3,3,2024-05-13 00:00:00,0,0,"[{'comment': 'ㅇㅎㅅ 교수님인가..ㅠ 허탈하겠다 고생했어…..ㅠ', 'l...",와 일찍 일어나서 부랴부랴 컴싸코 다 했는데 이번 주 과제네……………. 10주차는 ...,0
4,4,2024-05-13 00:00:00,0,0,[{'comment': '+지금 보니 다른 곳에 제출했었네요…\n이거 증거로 말씀드...,실험과목 레포트 다 써놓고 바보같이 제출을 안해서\n점수 0점 뜨고나서 알았는데\n...,0
...,...,...,...,...,...,...,...
12406,12406,2019-05-07,0,0,"[{'comment': '재수강입니다.', 'likes': 0}, {'comment...",제가 전학기에 성균논어 F받았는데요 이번 계절에 성균논어 들으면 최초수강취급되나요...,1
12407,12407,2019-05-07,0,0,[{'comment': '과마다 다른 사항이라 미컴 졸업논문 요건은 제가 모릅니다....,사회과학대학 졸업요건에서 미디어커뮤니케이션학과의 경우 논문대신 공인어학성적을 대신 ...,1
12408,12408,2019-05-07,0,0,"[{'comment': '전자공학자가 밤에 애인한테 보내는 문자는???', 'lik...",브로리님 재밌는얘기해주세요,1
12409,12409,2019-05-06,0,0,"[{'comment': '1. 네 2. 네', 'likes': 0}, {'comme...",브로리님\n계절 신청하고 싶은데 전년도 개설과목 찾아보니 전공이 거~의 안열리더라...,1


In [14]:
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [36]:
from transformers import BertModel
model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer
bert_model = BertModel.from_pretrained(model_checkpoint)
bert_model.to(device)
bert_model.eval()

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [44]:
# custom dataset class 정의
# 텍스터를 로딩 가능한 데이터로 변경하기 위한 작업.
# tokenizer를 거치면 token_type_ids를 통해 앞 시퀀스인지, 뒤 시퀀스인지 구별 가능

class text_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer # 원본 텍스트를 tokenize할 tokenizer
        self.data = dataframe # 원본 데이터
        self.comment_text = dataframe["merged"]
        self.targets = self.data["label"]
        self.max_len = max_len # tokenizer의 max length argument로 사용

    def __getitem__(self, idx):
        comment_text = str(self.comment_text[idx])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            add_special_tokens = True,
            max_length = self.max_len, # input_max_length
            padding = "max_length", # padding 할 길이
            truncation = True, # truncation 여부
            return_token_type_ids = True, # token_type_ids: 첫 번째 시퀀스(0)인지, 두 번째 시퀀스(1)인지 표시
        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"] #padding을 데이터 처리 하지 않기 위한
        token_type_ids = inputs["token_type_ids"]

        return {
            "ids": torch.tensor(ids, dtype = torch.long),
            "mask": torch.tensor(mask, dtype = torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype = torch.long),
            "targets": torch.tensor(self.targets[idx], dtype = torch.float)
        }

    def __len__(self):
        return len(self.comment_text)

In [45]:
random_seed = 10154315
train_size = 0.8 #훈련 데이터 사이즈 결정.
train_dataset = df.sample(frac = train_size, random_state = random_seed).reset_index(drop = True)
test_dataset = df.drop(train_dataset.index).reset_index(drop = True)

max_len = 300 #토큰화에 사용될 최대 허용 길이

trainDS = text_Dataset(train_dataset, tokenizer, max_len)
testDS = text_Dataset(test_dataset, tokenizer, max_len)

In [46]:
trainDL = DataLoader(trainDS, batch_size = 16, shuffle = True) # num_workers param also available
testDL = DataLoader(testDS, batch_size = 16, shuffle = True)

In [47]:
batch = next(iter(trainDL))
batch

{'ids': tensor([[   2,   80, 2013,  ...,    0,    0,    0],
         [   2, 9791, 2059,  ...,    0,    0,    0],
         [   2, 3651, 2822,  ...,    0,    0,    0],
         ...,
         [   2, 4224, 8683,  ...,    0,    0,    0],
         [   2, 5420, 1437,  ...,    0,    0,    0],
         [   2, 1517, 2145,  ...,    0,    0,    0]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'targets': tensor([1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])}

In [48]:
batch['mask'].size()

torch.Size([16, 300])

In [42]:
trainDS[0]
#id는 실제 데이터, 이중 0의 값은 패딩 값. mask는 패딩 값의 구분

{'ids': tensor([    2,  3855,  2595,  1891,  1141,  1169,  2223,  5971,    21,  3237,
          1437,  2311,    22,    18,  3801,    22,  3237,  1898,  2510,    22,
            18,  6448,    23,  3237,  1503,  2140,    21,    18,  6405,  9785,
          3737,  3628,  2154,  1897,  3551, 13964,  5110,  2073, 18119,    35,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [11]:
torch.cuda.is_available()

True

In [ ]:
train_dataset = df.sample(frac=train_size, random_state=random_seed).reset_index(drop=True)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)

In [66]:
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [71]:
# CNN 분류기 정의
from tqdm import tqdm

class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=768, out_channels=100, kernel_size=3, padding=1)
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(100, num_classes)
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch_size, embedding_dim, seq_length]
        x = F.relu(self.conv1(x))
        x = self.global_max_pool(x).squeeze(2)
        x = self.fc(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = CNNClassifier(num_classes=2)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 훈련 루프
for epoch in range(10):  # 에포크 수 조정 가능
    model.train()
    for batch in tqdm(trainDL):
        inputs = batch['ids'].to(device)
        masks = batch['mask'].to(device)
        labels = batch['targets'].to(device, dtype=torch.long)

        optimizer.zero_grad()

        # BERT를 통해 임베딩 추출
        with torch.no_grad():
            embeddings = bert_model(inputs, attention_mask=masks).last_hidden_state

        # CNN을 통한 분류
        outputs = model(embeddings)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

100%|██████████| 621/621 [02:44<00:00,  3.77it/s]


Epoch 1, Loss: 0.3738429844379425


100%|██████████| 621/621 [02:46<00:00,  3.73it/s]


Epoch 2, Loss: 0.4357847571372986


100%|██████████| 621/621 [02:46<00:00,  3.73it/s]


...

Epoch 8, Loss: 0.145547553896904


100%|██████████| 621/621 [02:46<00:00,  3.73it/s]


Epoch 9, Loss: 0.27007293701171875


100%|██████████| 621/621 [02:46<00:00,  3.74it/s]

Epoch 10, Loss: 0.059963107109069824


In [77]:
def validation():
    model.eval()  # 활성화되지 않은 계층을 위해서
    fin_targets = []  # 실제 레이블
    fin_outputs = []  # 출력 레이블
    with torch.no_grad():
        for _, data in enumerate(testDL):  # 배치 단위로 데이터셋 가져옴
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            with torch.no_grad():
                embeddings = bert_model(ids, attention_mask=mask).last_hidden_state
            outputs = model(embeddings)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(outputs.argmax(dim=1).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets  # 결과 반환

# 검증 수행 및 점수 계산
outputs, targets = validation()

# 이진 분류를 위한 출력 변환
outputs = np.array(outputs)
targets = np.array(targets)

accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.9858984689766318
F1 Score (Micro) = 0.9858984689766318
F1 Score (Macro) = 0.9260073166472891
